In [ ]:
from cubelib.stac_eco import Stac_eco
from cubelib.fm_map import Fmap

In [ ]:
import pandas as pd
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 2000)

In [ ]:
#! cp ../2_Gridding_For_Scale/*.geojson .
! ls *.geojson

In [ ]:
geojson_file = 'one_tile.geojson'
se = Stac_eco(geojson_file)

In [ ]:
se

In [ ]:
se.set_collection('landsat-c2ard-sr')

In [ ]:
se

In [ ]:
fm = Fmap()
fm.sat_geojson(geojson_file)

In [ ]:
dates="2020-04-01/2020-10-31"

In [ ]:
search_object_eco = se.search(dates)

In [ ]:
number_of_matched_scenes = search_object_eco.matched()

In [ ]:
print(f"I found {number_of_matched_scenes} Scenes yay!")

In [ ]:
so = search_object_eco

In [ ]:
gdf1 = se.items_gdf(so)

In [ ]:
#gdf1

In [ ]:
gdf1.T

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
gdf1['stac_extensions']

In [ ]:
se.plot_polygons(so)

In [ ]:
gdf1['properties.landsat:grid_vertical']

In [ ]:
gdf1['properties.landsat:grid_horizontal']

In [ ]:
gdf2 = gdf1[gdf1['properties.landsat:grid_horizontal']=='29']

In [ ]:
gdf2.T

In [ ]:
gdf3 = gdf2[gdf2['properties.landsat:grid_vertical']=='03']

In [ ]:
gdf3[['properties.landsat:grid_horizontal', 'properties.landsat:grid_vertical']]

In [ ]:
len(gdf3[['properties.landsat:grid_horizontal', 'properties.landsat:grid_vertical']])

In [ ]:
dir(se)

In [ ]:
se.df_assets(so)

In [ ]:
import boto3
from rasterio.session import AWSSession
aws_session = AWSSession(boto3.Session(), requester_pays=True)

In [ ]:
import rasterio as rio
import xarray as xr
def create_dataset(row, bands = ['Swirs', 'Green'], chunks = {'band': 1, 'x':2048, 'y':2048}):
    datasets = []
    with rio.Env(aws_session):
        for band in bands:
            print(row[band]['href'])
            url = row[band]['href']
            url = url.replace('usgs-landsat', 'usgs-landsat-ard')
            #da = xr.open_rasterio(url, chunks = chunks)
            da = xr.open_rasterio(url)
            daSub=da
#             daSub = da.sel(x=slice(ll_corner[0], ur_corner[0]), y=slice(ur_corner[1], ll_corner[1]))
            daSub = daSub.squeeze().drop(labels='band')
            DS = daSub.to_dataset(name = band)
            datasets.append(DS)
        DS = xr.merge(datasets)
        return DS

In [ ]:
def asset_gdf(my_gdf,bands):
    #print(my_gdf.keys)
    i_dict_array = []
    for i,item in my_gdf.iterrows():
            i_dict ={}
            print(item.id)
            i_dict['id'] = item.id
            for band in bands:
                href = f'assets.{band}.href'
                #print(item[href])
                i_dict[band] = {'band': band,
                 'href': item[href]
                }
            i_dict_array.append(i_dict)
    print(i_dict_array)
    new_gdf = pd.DataFrame(i_dict_array)
    return new_gdf

In [ ]:
gdf3

In [ ]:
bands=['blue','green','red','nir08','qa_pixel']
gdf4=asset_gdf(gdf3,bands)

In [ ]:
gdf4.id

In [ ]:
datasets = []
for i,row in gdf4.iterrows():
    try:
        print('loading....', row.id)
        
        ds = create_dataset(row,bands)
        datasets.append(ds)
    except Exception as e:
        print('Error loading, skipping')
        print(e)

In [ ]:
! aws s3 ls --request-payer requester s3://usgs-landsat/collection02/oli-tirs/2020/CU/029/003/LC08_CU_029003_20200419_20210504_02/LC08_CU_029003_20200419_20210504_02_SR_B2.TIF

In [ ]:
! aws s3 ls --request-payer requester s3://usgs-landsat/collection02/oli-tirs/2020/CU/029/003/

In [ ]:
! aws s3 ls --request-payer requester s3://usgs-landsat-ard/collection02/oli-tirs/2020/CU/029/003/

In [ ]:
datasets

In [ ]:
! date

In [ ]:
gdf3

In [ ]:
gdf3.keys()

In [ ]:
gdf3['properties.start_datetime'].tolist()

In [ ]:
len(gdf3)

In [ ]:
gdf3.index.tolist()

In [ ]:
from datetime import datetime
my_date_list = gdf3.index.tolist()
my_str_date_list=[]
for dt in my_date_list:
    print(dt)
    str_dt = dt.strftime('%Y-%m-%d')
    print(str_dt)
    my_str_date_list.append(str_dt)

In [ ]:
DS = xr.concat(datasets, dim= pd.DatetimeIndex(my_str_date_list, name='time'))

In [ ]:
print('Dataset Size (Gb): ', DS.nbytes/1e9)

In [ ]:
DS

In [ ]:
DS['red'].isel(time=0).plot()

In [ ]:
DS['red'][1].plot()

In [ ]:
DS['red'][15].plot()

In [ ]:
ds_mini = DS.isel(x=slice(0,5000,10), y=slice(0,5000,10))

In [ ]:
ds_mini

In [ ]:
%matplotlib inline

display_color = 'blue'
# 
#ds_mini[display_color].plot.imshow('x','y', col='time', col_wrap=6, cmap='viridis')
ds_mini[display_color].plot.imshow('x','y', col='time', col_wrap=6, cmap='viridis', vmin=7000, vmax=19000)

In [ ]:
ds_mini.hvplot()

In [ ]:
ds_mini['red'][0].hvplot.image(rasterize=True)

In [ ]:
ds_mini['red'][0].plot()

In [ ]:
d2 = ds_mini.transpose('time', 'y', 'x')

In [ ]:
d2['red'].hvplot.image(rasterize=True)

In [ ]:
d2

In [ ]:
dir(d2)

In [ ]:
d2.swap_dims({'time')

In [ ]:
help(d2.swap_dims)

In [ ]:
help(d2.rename)

In [ ]:
d2.swap_dims({'time':'x'})

In [ ]:
d2.dims

In [ ]:
d2.drop_dims()

In [ ]:
help(d2.drop_dims)

In [ ]:
d2['red'].hvplot.image(rasterize=True,  x='x', y='y', width=600, height=400, cmap='viridis', clim=(4000,20000))

In [ ]:
help(d2['red'].hvplot.image)

In [ ]:
d2['qa_pixel'].hvplot.image(rasterize=True,  x='x', y='y', width=600, height=400, cmap='viridis')